In [1]:
import pandas as pd 
import gurobipy as gp 
from gurobipy import GRB
import warnings 

warnings.filterwarnings('ignore')

In [2]:
players = pd.read_csv('MLB_Assignment/players2.csv', header = 0)

In [3]:
players.columns

Index(['Unnamed: 0', 'Name', 'Normalized_WAR', 'off', 'st_pit', 'rl_pit',
       'team', 'salary', 'position', 'Arizona', 'Atlanta', 'Baltimore',
       'Boston', 'Chic. Cubs', 'Chic. White Sox', 'Chicago Cubs', 'Cincinnati',
       'Cleveland', 'Colorado', 'Detroit', 'Houston', 'Kansas City',
       'L.A. Angels', 'L.A. Dodgers', 'Miami', 'Milwaukee', 'Minnesota',
       'N.Y. Mets', 'N.Y. Yankees', 'Oakland', 'Philadelphia', 'Pittsburgh',
       'San Diego', 'San Francisco', 'Seattle', 'St. Louis', 'Tampa Bay',
       'Texas', 'Toronto', 'Washington', '0', '1B', '2B', '3B', 'C', 'CF',
       'DH', 'INF', 'LF', 'OF', 'RF', 'RP', 'SP', 'SS', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', '1B = 2.0', '2B = 2.26', '3B= 2.6'],
      dtype='object')

In [4]:
players['10_mil'] = 0
players['5_10_mil'] = 0
players['2_5_mil'] = 0 
players['0_2_mil'] = 0 

for i in range(len(players)):
    if players.salary[i] >= 10000000:
        players['10_mil'][i] = 1
    elif 5000000 <= players.salary[i] <= 10000000:
        players['5_10_mil'][i] = 1
    elif 2000000 <= players.salary[i] <= 5000000:
        players['2_5_mil'][i] = 1 
    else: 
        players['0_2_mil'][i] = 1

In [5]:
players

,Unnamed: 0,Name,Normalized_WAR,off,st_pit,rl_pit,team,salary,position,Arizona,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,1B = 2.0,2B = 2.26,3B= 2.6,10_mil,5_10_mil,2_5_mil,0_2_mil
0,0,Trea Turner,6.4,1,0,0,Washington,13000000,SS,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
1,1,Vladimir Guerrero Jr.,5.8,1,0,0,Toronto,605400,OF,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
2,2,Bryce Harper,6.6,1,0,0,Philadelphia,27538462,OF,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
3,3,Marcus Semien,5.5,1,0,0,Toronto,18000000,2B,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
4,4,Juan Soto,6.1,1,0,0,Washington,8500000,OF,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,639,Albert Abreu,-1.0,0,0,1,N.Y. Yankees,575000,RP,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
640,640,Kyle Keller,-1.1,0,0,1,Pittsburgh,575000,RP,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
641,641,Bryan Garcia,-1.4,0,0,1,Detroit,576600,RP,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
642,642,Alex Young,-1.2,0,0,1,Arizona,603000,RP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1


In [6]:
m = gp.Model('Player_selection')

Academic license - for non-commercial use only - expires 2022-09-25
Using license file /Users/youssefragab/gurobi.lic


In [7]:
x = m.addVars(players.index, vtype = GRB.BINARY, name = 'x')

In [8]:
budget = 132000000

In [9]:
# Constraints for number of pitchers 
m.addConstr(sum(x[i] for i in players.index if players.SP[i] == 1) == 5)
m.addConstr(sum(x[i] for i in players.index if players.RP[i] == 1) == 7)

# Contstraint for number of catchers
m.addConstr(sum(x[i] for i in players.index if players['C'][i] == 1) == 2)

# Constraint for number of basemen 
m.addConstr(sum(x[i] for i in players.index if players['1B'][i] == 1) >= 1)
m.addConstr(sum(x[i] for i in players.index if players['2B'][i] == 1) >= 1)
m.addConstr(sum(x[i] for i in players.index if players['3B'][i] == 1) >= 1)
m.addConstr(sum(x[i] for i in players.index if players['SS'][i] == 1) == 3)

# Constraints for number of outfielders
m.addConstr(sum(x[i] for i in players.index if players['LF'][i] == 1) >= 1)
m.addConstr(sum(x[i] for i in players.index if players['CF'][i] == 1) >= 1)
m.addConstr(sum(x[i] for i in players.index if players['RF'][i] == 1) >= 1)

# Constraint for total number of players 
m.addConstr(sum(x[i] for i in players.index) == 25 )

<gurobi.Constr *Awaiting Model Update*>

In [10]:
for j in set(list(players.team)):
    m.addConstr(sum(x[i] for i in players.index if players[j][i] == 1) <= 2)

In [11]:
# Constraints for number of players in each price bracket 
m.addConstr(sum(x[i] for i in players.index if players['10_mil'][i] == 1) <= 5)

m.addConstr(sum(x[i] for i in players.index if players['5_10_mil'][i] == 1) <= 4)

m.addConstr(sum(x[i] for i in players.index if players['2_5_mil'][i] == 1) <= 4)

<gurobi.Constr *Awaiting Model Update*>

In [12]:
# Total budget constraint 
m.addConstr(sum(x[i] * players.salary[i] for i in players.index) <= budget)

<gurobi.Constr *Awaiting Model Update*>

In [13]:
# Setting up objective function
obj = sum(x[i] * players.Normalized_WAR[i] for i in players.index)

In [14]:
m.setObjective(obj, GRB.MAXIMIZE)

In [15]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 46 rows, 644 columns and 2771 nonzeros
Model fingerprint: 0xeed4387c
Variable types: 0 continuous, 644 integer (644 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+07]
  Objective range  [1e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Found heuristic solution: objective 30.6000000
Presolve removed 0 rows and 14 columns
Presolve time: 0.00s
Presolved: 46 rows, 630 columns, 2282 nonzeros
Found heuristic solution: objective 117.2000000
Variable types: 0 continuous, 630 integer (626 binary)

Root relaxation: objective 1.236119e+02, 68 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  123.61193    0    6  117.20000  123.61193  5.47%     -    0s
H    0 

In [16]:
team_list = []
for i in players.index:
    if m.getVars()[i].x == 1:
        team_list.append(players.loc[i, ['Name', 'position', 'team', 'salary', 'Normalized_WAR']])

In [17]:
team_df =pd.DataFrame(team_list)

In [18]:
print('Selected Team:')
display(team_df)

Selected Team:


,Name,position,team,salary,Normalized_WAR
0,Trea Turner,SS,Washington,13000000,6.4
5,Jose Ramirez,3B,Cleveland,9400000,5.9
6,Fernando Tatis Jr.,SS,San Diego,1714286,6.7
7,Carlos Correa,SS,Houston,11700000,5.4
12,Tyler O'Neill,OF,St. Louis,594700,6.0
14,Brandon Lowe,2B,Tampa Bay,2500000,5.1
25,Will Smith,C,L.A. Dodgers,590500,5.5
26,Mike Zunino,C,Tampa Bay,2000000,7.2
34,Ronald Acuna Jr.,OF,Atlanta,5000000,7.0
81,Adolis Garcia,RF,Texas,575000,2.8


In [19]:
team_df.to_clipboard()